# IBM Data Science Professional certificate Capstone project - Toronto neighbourhoods

## 1. Scrape data from Toronto wiki

In [2]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 993kB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
rg = requests.get(url)

In [4]:
rg

<Response [200]>

In [5]:
soup = BeautifulSoup(rg.content, 'lxml')

In [6]:
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [7]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
df.shape

(103, 3)

## 2. Get the latitude and the longitude coordinates of each neighborhood

In [11]:
import requests 
import pandas as pd 
import numpy as np
import random 

!pip install geopy
from geopy.geocoders import Nominatim # converts an address into latitude and longitude

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 1.1MB/s eta 0:00:01
Folium installed
Libraries imported.


In [12]:
CLIENT_ID = 'S3BUNLSRTBBYU4PETEL4YEOVKRJVUYGISOSUQ4AWUC2EXAVU' # your Foursquare ID
CLIENT_SECRET = 'SHB3XXUAMJTABTHDSVV0CG1PJOTCTYCHJQMD03ATIW3IA0RG' # your Foursquare Secret
ACCESS_TOKEN = '310L4WERKIK2NV5QLYBB1BHRP1OK2Q2BZ42B1NLTOKPLFTQV' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3BUNLSRTBBYU4PETEL4YEOVKRJVUYGISOSUQ4AWUC2EXAVU
CLIENT_SECRET:SHB3XXUAMJTABTHDSVV0CG1PJOTCTYCHJQMD03ATIW3IA0RG


In [30]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
coordinates = pd.read_csv(url)

coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']

coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
coordinates.shape

(103, 3)

In [31]:
df_coord = df.join(coordinates.set_index('PostalCode'), on='PostalCode', how='inner')
df_coord.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [32]:
df_coord.shape

(103, 5)

## 3. Explore and cluster the neighborhoods in Toronto

In [33]:
print('The city of Toronto has {} boroughs and {} neighborhoods.'.format(len(df_coord['Borough'].unique()),df_coord.shape[0]))

The city of Toronto has 15 boroughs and 103 neighborhoods.


### Let us explore the Borough of 'Downtown Toronto'

In [41]:
df_downtown = df_coord[df_coord['Borough'] == 'Downtown Toronto']
df_downtown.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [38]:
print('The Borough of Downtown Toronto has {} neighborhoods.'.format(df_downtown.shape[0]))

The Borough of Downtown Toronto has 17 neighborhoods.


we will explore the neighborhood of Regent Park, Harbourfront

In [45]:
regpark = df_downtown[df_downtown['Neighborhood'] == 'Regent Park, Harbourfront']
regpark 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


### Construct a URL to send a request to the API to find parks near Regent Park

In [57]:
latitude = float(regpark['Latitude'])
longitude = float(regpark['Longitude'])
search_query = 'parks'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S3BUNLSRTBBYU4PETEL4YEOVKRJVUYGISOSUQ4AWUC2EXAVU&client_secret=SHB3XXUAMJTABTHDSVV0CG1PJOTCTYCHJQMD03ATIW3IA0RG&ll=43.6542599,-79.3606359&oauth_token=310L4WERKIK2NV5QLYBB1BHRP1OK2Q2BZ42B1NLTOKPLFTQV&v=20180604&query=parks&radius=500&limit=30'

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b256585d2a2d6d2a25d3c9'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4eef45335c5c794ad4305a1d',
    'name': 'Regent Park School of Music',
    'location': {'address': '534 Queen Street East',
     'lat': 43.65691161146316,
     'lng': -79.3571683258484,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65691161146316,
       'lng': -79.3571683258484}],
     'distance': 406,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['534 Queen Street East', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e5931735',
      'name': 'Music Venue',
      'pluralName': 'Music Venues',
      'shortName': 'Music Venue',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1622300248',
    

In [59]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4eef45335c5c794ad4305a1d,Regent Park School of Music,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",v-1622300248,False,534 Queen Street East,43.656912,-79.357168,"[{'label': 'display', 'lat': 43.65691161146316...",406,CA,Toronto,ON,Canada,"[534 Queen Street East, Toronto ON, Canada]"
1,4f507596e4b0a3428a2f3c98,Regent Park South Rink,"[{'id': '4bf58dd8d48988d168941735', 'name': 'S...",v-1622300248,False,NaN,43.659131,-79.364923,"[{'label': 'display', 'lat': 43.65913102287877...",642,CA,Toronto,ON,Canada,"[Toronto ON, Canada]"


In [60]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,id
0,Regent Park School of Music,Music Venue,534 Queen Street East,43.656912,-79.357168,"[{'label': 'display', 'lat': 43.65691161146316...",406,CA,Toronto,ON,Canada,"[534 Queen Street East, Toronto ON, Canada]",4eef45335c5c794ad4305a1d
1,Regent Park South Rink,Skating Rink,NaN,43.659131,-79.364923,"[{'label': 'display', 'lat': 43.65913102287877...",642,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4f507596e4b0a3428a2f3c98


#### Lets visualize the parks near Regents Park

In [61]:
dataframe_filtered.name

0    Regent Park School of Music
1         Regent Park South Rink
Name: name, dtype: object

In [62]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [63]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent Regent Park, Harbourfront
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Regent Park, Harbourfront',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Thus we can find the number of parks in a neighbourhood

### Regent Park, Harbourfront